# Ridge Regression with Gradient Descent

In this notebook, we will implement ridge regression via gradient descent. We will:
* Convert an SFrame into a Numpy array
* Write a predict_output() function using Numpy
* Write a numpy function to compute the derivative of the regression weights with respect to a single feature
* Write gradient descent function to compute the regression weights given an initial weight vector, step size, tolerance, and L2 penalty

Make sure you have the latest version of Turi Create

In [67]:
import turicreate
import numpy as np
import matplotlib.pyplot as plt

# Load in House Sales Data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [21]:
sales = turicreate.SFrame(r'\home_data.sframe')
sales.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650.0,1.0,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242.0,2.0,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000.0,1.0,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000.0,1.0,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080.0,1.0,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930.0,1.0,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819.0,2.0,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711.0,1.0,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470.0,1.0,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560.0,2.0,0


# Convert to Numpy Array

Although SFrames offer a number of benefits to users (especially when using Big Data and built-in Turi Create functions), in order to understand the details of the implementation of algorithms it's important to work with a library that allows for direct (and optimized) matrix operations. Numpy is a Python solution to work with matrices (or any multi-dimensional "array").

Recall that the predicted value given the weights and the features is just the dot product between the feature and weight vector. Similarly, if we put all of the features row-by-row in a matrix then the predicted value for *all* the observations can be computed by right multiplying the "feature matrix" by the "weight vector". 
 
Now we will write a function that will accept an SFrame, a list of feature names (['sqft_living', 'sqft_living_15']) and a target feature ('price') and will return two things:
* A numpy matrix whose columns are the desired features plus a constant column (this is how we create an intercept)
* A numpy array containing the values of the output

In [22]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe=data_sframe[features]
    
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray=data_sframe[output]

    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

We now define an additional function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [23]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions=np.dot(feature_matrix, weights)

    return(predictions)

# Computing the Derivative

We are now going to move to computing the derivative of the regression cost function. Recall that the cost function is the sum over the data points of the squared difference between an observed output and a predicted output, plus the L2 penalty term. Since the derivative of a sum is the sum of the derivatives, we can take the derivative of the first part (the RSS) as we did in the notebook for the unregularized case and add the derivative of the regularization part. 

**We will not regularize the constant.**  As a result, the derivative is just twice the sum of the errors (without the `2*l2_penalty*w[0]` term).

In [24]:
def feature_derivative_ridge(errors, feature, weight, l2_penalty, feature_is_constant):
    # If feature_is_constant is True, derivative is twice the dot product of errors and feature
    if feature_is_constant==True:
        derivative=2*np.dot(errors,feature)
    # Otherwise, derivative is twice the dot product plus 2*l2_penalty*weight
    else:
        derivative=2*np.dot(errors,feature)+2*l2_penalty*weight
    
    return derivative

# Gradient Descent

Now we will write a function that performs a gradient descent. The basic premise is simple. Given a starting point we update the current weights by moving in the negative gradient direction. Recall that the gradient is the direction of *increase* and therefore the negative gradient is the direction of *decrease* and we're trying to *minimize* a cost function. 

The amount by which we move in the negative gradient *direction*  is called the 'step size'. We stop when we are 'sufficiently close' to the optimum. Unlike in the *Linear Regression with Gradient Descent* notebook, this time we will set a **maximum number of iterations** and take gradient steps until we reach this maximum number. If no maximum number is supplied, the maximum should be set 100 by default. (Use default parameter values in Python.)

In [25]:
def ridge_regression_gradient_descent(feature_matrix, output, initial_weights, step_size, l2_penalty, max_iterations=100):
    print 'Starting gradient descent with l2_penalty = ' + str(l2_penalty)
    
    weights = np.array(initial_weights) # make sure it's a numpy array
    iteration = 0 # iteration counter
    print_frequency = 1  # for adjusting frequency of debugging output
    
    #while not reached maximum number of iterations:
    while iteration<max_iterations:
        iteration += 1  # increment iteration counter
        ### === code section for adjusting frequency of debugging output. ===
        if iteration == 10:
            print_frequency = 10
        if iteration == 100:
            print_frequency = 100
        if iteration%print_frequency==0:
            print('Iteration = ' + str(iteration))
        ### === end code section ===
        
        # compute the predictions based on feature_matrix and weights using your predict_output() function
        predictions=predict_output(feature_matrix, weights)
        # compute the errors as predictions - output
        errors=predictions-output
        # from time to time, print the value of the cost function
        if iteration%print_frequency==0:
            print 'Cost function = ', str(np.dot(errors,errors) + l2_penalty*(np.dot(weights,weights) - weights[0]**2))
        
        for i in xrange(len(weights)): # loop over each weight
            # Recall that feature_matrix[:,i] is the feature column associated with weights[i]
            # compute the derivative for weight[i].
            if i==0:
                derivative = feature_derivative_ridge(errors, feature_matrix[:,i], weights[i], l2_penalty, True)
            
            else:
                derivative = feature_derivative_ridge(errors, feature_matrix[:,i], weights[i], l2_penalty, False)
           
        # subtract the step size times the derivative from the current weight
            weights[i]=weights[i]-step_size*derivative
            
    print 'Done with gradient descent at iteration ', iteration
    print 'Learned weights = ', str(weights)
    return weights

The L2 penalty gets its name because it causes weights to have small L2 norms than otherwise. Let's see how large weights get penalized. Let us consider a simple model with 1 feature:

Let us split the dataset into training set and test set. Make sure to use `seed=0`:

In [73]:
model_features = ['sqft_living', 'sqft_living15'] # sqft_living15 is the average squarefeet for the nearest 15 neighbors. 
my_output = 'price'
initial_weights = np.array([-95000.0,0.0,0.0])
step_size = 1e-12
max_iterations = 5000000
l2_penalty=0.1

In [74]:
(feature_matrix, output) = get_numpy_data(sales, model_features, my_output)

algorithm_weights = ridge_regression_gradient_descent(feature_matrix, output, 
                                                               initial_weights, step_size, 
                                                               l2_penalty, max_iterations)

Starting gradient descent with l2_penalty = 0.1
Iteration = 1
Cost function =  1.1630238210640734e+16
Iteration = 2
Cost function =  5065122841183831.0
Iteration = 3
Cost function =  2756228958630935.0
Iteration = 4
Cost function =  1943925718103910.0
Iteration = 5
Cost function =  1657864189498229.0
Iteration = 6
Cost function =  1556847572624859.5
Iteration = 7
Cost function =  1520903235994641.5
Iteration = 8
Cost function =  1507845987817400.2
Iteration = 9
Cost function =  1502842466806694.2
Iteration = 10
Cost function =  1500676837344376.0
Iteration = 20
Cost function =  1494013729744420.2
Iteration = 30
Cost function =  1488979851759300.2
Iteration = 40
Cost function =  1484634356408288.5
Iteration = 50
Cost function =  1480883089482148.2
Iteration = 60
Cost function =  1477644792713430.8
Iteration = 70
Cost function =  1474849319565564.5
Iteration = 80
Cost function =  1472436115749248.0
Iteration = 90
Cost function =  1470352907522514.5
Iteration = 100
Cost function =  146855

Iteration = 14800
Cost function =  1457201949851412.5
Iteration = 14900
Cost function =  1457201949819948.5
Iteration = 15000
Cost function =  1457201949788484.8
Iteration = 15100
Cost function =  1457201949757020.5
Iteration = 15200
Cost function =  1457201949725557.0
Iteration = 15300
Cost function =  1457201949694093.2
Iteration = 15400
Cost function =  1457201949662629.8
Iteration = 15500
Cost function =  1457201949631165.8
Iteration = 15600
Cost function =  1457201949599702.2
Iteration = 15700
Cost function =  1457201949568238.5
Iteration = 15800
Cost function =  1457201949536774.8
Iteration = 15900
Cost function =  1457201949505311.2
Iteration = 16000
Cost function =  1457201949473847.8
Iteration = 16100
Cost function =  1457201949442384.2
Iteration = 16200
Cost function =  1457201949410921.0
Iteration = 16300
Cost function =  1457201949379457.5
Iteration = 16400
Cost function =  1457201949347994.0
Iteration = 16500
Cost function =  1457201949316530.5
Iteration = 16600
Cost funct

Iteration = 30900
Cost function =  1457201944786101.5
Iteration = 31000
Cost function =  1457201944754642.2
Iteration = 31100
Cost function =  1457201944723182.5
Iteration = 31200
Cost function =  1457201944691723.8
Iteration = 31300
Cost function =  1457201944660264.8
Iteration = 31400
Cost function =  1457201944628805.8
Iteration = 31500
Cost function =  1457201944597346.8
Iteration = 31600
Cost function =  1457201944565887.2
Iteration = 31700
Cost function =  1457201944534428.2
Iteration = 31800
Cost function =  1457201944502969.5
Iteration = 31900
Cost function =  1457201944471510.5
Iteration = 32000
Cost function =  1457201944440051.5
Iteration = 32100
Cost function =  1457201944408592.8
Iteration = 32200
Cost function =  1457201944377133.2
Iteration = 32300
Cost function =  1457201944345675.0
Iteration = 32400
Cost function =  1457201944314216.2
Iteration = 32500
Cost function =  1457201944282757.5
Iteration = 32600
Cost function =  1457201944251298.8
Iteration = 32700
Cost funct

Iteration = 46200
Cost function =  1457201939973176.8
Iteration = 46300
Cost function =  1457201939941722.2
Iteration = 46400
Cost function =  1457201939910267.2
Iteration = 46500
Cost function =  1457201939878812.8
Iteration = 46600
Cost function =  1457201939847358.0
Iteration = 46700
Cost function =  1457201939815902.8
Iteration = 46800
Cost function =  1457201939784448.2
Iteration = 46900
Cost function =  1457201939752994.0
Iteration = 47000
Cost function =  1457201939721539.5
Iteration = 47100
Cost function =  1457201939690084.5
Iteration = 47200
Cost function =  1457201939658630.0
Iteration = 47300
Cost function =  1457201939627175.8
Iteration = 47400
Cost function =  1457201939595721.2
Iteration = 47500
Cost function =  1457201939564266.8
Iteration = 47600
Cost function =  1457201939532812.2
Iteration = 47700
Cost function =  1457201939501357.8
Iteration = 47800
Cost function =  1457201939469903.2
Iteration = 47900
Cost function =  1457201939438449.2
Iteration = 48000
Cost funct

Iteration = 63600
Cost function =  1457201934500476.8
Iteration = 63700
Cost function =  1457201934469026.8
Iteration = 63800
Cost function =  1457201934437577.2
Iteration = 63900
Cost function =  1457201934406127.5
Iteration = 64000
Cost function =  1457201934374677.5
Iteration = 64100
Cost function =  1457201934343228.2
Iteration = 64200
Cost function =  1457201934311778.5
Iteration = 64300
Cost function =  1457201934280329.0
Iteration = 64400
Cost function =  1457201934248879.2
Iteration = 64500
Cost function =  1457201934217429.5
Iteration = 64600
Cost function =  1457201934185980.2
Iteration = 64700
Cost function =  1457201934154531.0
Iteration = 64800
Cost function =  1457201934123081.2
Iteration = 64900
Cost function =  1457201934091631.8
Iteration = 65000
Cost function =  1457201934060182.2
Iteration = 65100
Cost function =  1457201934028733.2
Iteration = 65200
Cost function =  1457201933997283.8
Iteration = 65300
Cost function =  1457201933965834.2
Iteration = 65400
Cost funct

...

Iteration = 4932000
Cost function =  1457200437123098.5
Iteration = 4932100
Cost function =  1457200437093024.0
Iteration = 4932200
Cost function =  1457200437062949.5
Iteration = 4932300
Cost function =  1457200437032875.2
Iteration = 4932400
Cost function =  1457200437002800.5
Iteration = 4932500
Cost function =  1457200436972726.0
Iteration = 4932600
Cost function =  1457200436942651.8
Iteration = 4932700
Cost function =  1457200436912577.0
Iteration = 4932800
Cost function =  1457200436882503.0
Iteration = 4932900
Cost function =  1457200436852428.8
Iteration = 4933000
Cost function =  1457200436822354.5
Iteration = 4933100
Cost function =  1457200436792279.8
Iteration = 4933200
Cost function =  1457200436762205.8
Iteration = 4933300
Cost function =  1457200436732131.2
Iteration = 4933400
Cost function =  1457200436702057.0
Iteration = 4933500
Cost function =  1457200436671982.5
Iteration = 4933600
Cost function =  1457200436641908.5
Iteration = 4933700
Cost function =  14572004366

Iteration = 4947400
Cost function =  1457200432491937.0
Iteration = 4947500
Cost function =  1457200432461866.5
Iteration = 4947600
Cost function =  1457200432431795.8
Iteration = 4947700
Cost function =  1457200432401725.5
Iteration = 4947800
Cost function =  1457200432371655.5
Iteration = 4947900
Cost function =  1457200432341585.2
Iteration = 4948000
Cost function =  1457200432311515.0
Iteration = 4948100
Cost function =  1457200432281445.0
Iteration = 4948200
Cost function =  1457200432251374.8
Iteration = 4948300
Cost function =  1457200432221304.5
Iteration = 4948400
Cost function =  1457200432191234.5
Iteration = 4948500
Cost function =  1457200432161164.2
Iteration = 4948600
Cost function =  1457200432131094.2
Iteration = 4948700
Cost function =  1457200432101024.2
Iteration = 4948800
Cost function =  1457200432070954.5
Iteration = 4948900
Cost function =  1457200432040884.5
Iteration = 4949000
Cost function =  1457200432010814.5
Iteration = 4949100
Cost function =  14572004319

Iteration = 4962400
Cost function =  1457200427981694.0
Iteration = 4962500
Cost function =  1457200427951628.2
Iteration = 4962600
Cost function =  1457200427921562.0
Iteration = 4962700
Cost function =  1457200427891495.8
Iteration = 4962800
Cost function =  1457200427861429.5
Iteration = 4962900
Cost function =  1457200427831363.2
Iteration = 4963000
Cost function =  1457200427801297.5
Iteration = 4963100
Cost function =  1457200427771231.5
Iteration = 4963200
Cost function =  1457200427741165.5
Iteration = 4963300
Cost function =  1457200427711099.2
Iteration = 4963400
Cost function =  1457200427681033.5
Iteration = 4963500
Cost function =  1457200427650967.5
Iteration = 4963600
Cost function =  1457200427620901.5
Iteration = 4963700
Cost function =  1457200427590835.8
Iteration = 4963800
Cost function =  1457200427560770.0
Iteration = 4963900
Cost function =  1457200427530704.2
Iteration = 4964000
Cost function =  1457200427500638.2
Iteration = 4964100
Cost function =  14572004274

Iteration = 4977800
Cost function =  1457200423351825.0
Iteration = 4977900
Cost function =  1457200423321762.8
Iteration = 4978000
Cost function =  1457200423291701.0
Iteration = 4978100
Cost function =  1457200423261639.0
Iteration = 4978200
Cost function =  1457200423231577.0
Iteration = 4978300
Cost function =  1457200423201515.5
Iteration = 4978400
Cost function =  1457200423171453.5
Iteration = 4978500
Cost function =  1457200423141392.0
Iteration = 4978600
Cost function =  1457200423111330.2
Iteration = 4978700
Cost function =  1457200423081268.0
Iteration = 4978800
Cost function =  1457200423051206.5
Iteration = 4978900
Cost function =  1457200423021145.0
Iteration = 4979000
Cost function =  1457200422991083.8
Iteration = 4979100
Cost function =  1457200422961021.8
Iteration = 4979200
Cost function =  1457200422930960.2
Iteration = 4979300
Cost function =  1457200422900898.5
Iteration = 4979400
Cost function =  1457200422870837.2
Iteration = 4979500
Cost function =  14572004228

Iteration = 4992800
Cost function =  1457200418842841.2
Iteration = 4992900
Cost function =  1457200418812783.8
Iteration = 4993000
Cost function =  1457200418782725.8
Iteration = 4993100
Cost function =  1457200418752668.0
Iteration = 4993200
Cost function =  1457200418722610.2
Iteration = 4993300
Cost function =  1457200418692553.0
Iteration = 4993400
Cost function =  1457200418662495.2
Iteration = 4993500
Cost function =  1457200418632437.5
Iteration = 4993600
Cost function =  1457200418602379.5
Iteration = 4993700
Cost function =  1457200418572322.2
Iteration = 4993800
Cost function =  1457200418542264.8
Iteration = 4993900
Cost function =  1457200418512207.2
Iteration = 4994000
Cost function =  1457200418482149.5
Iteration = 4994100
Cost function =  1457200418452092.2
Iteration = 4994200
Cost function =  1457200418422035.0
Iteration = 4994300
Cost function =  1457200418391977.2
Iteration = 4994400
Cost function =  1457200418361919.5
Iteration = 4994500
Cost function =  14572004183

In [70]:
model_ridge = turicreate.linear_regression.create(sales, 
                                                  target=my_output,
                                                  features=model_features,
                                                  verbose=False,
                                                  max_iterations=max_iterations,
                                                  l2_penalty=l2_penalty)
model_ridge.coefficients

name,index,value,stderr
(intercept),None,-101727.872888,5582.10093048
sqft_living,None,243.652343269,3.00802848151
sqft_living15,None,68.1329755926,4.04549088094


As you can see, the coefficient weights on sqft_living and sqft_living15 estimated using Turi Create are very similar to the weights estimated by our algorithm!